# Connect to Audit Logs in Sigma to get Workbook Usage Metrics

### Prequisite Steps
1. Enable Audit Logs
2. Create workbook that matches output (Sully will show how)
3. Save the workbook Id and the node Id of this table (both accessible through URL when table selected)
4. Create a Sigma API token with Client ID and Secret. Save these in the .env file

## Once we've done these steps, we now have everything we need in order to access this information programmatically

# Steps to access this data via APIs
1. Auth into Sigma APIs with client id and token
2. Use the export API to choose which data we want to send
3. Use the download API to download this information

In [8]:
import requests
import os

# Parse the .env file
with open('.env') as f:
    for line in f:
        key, value = line.strip().split('=')
        os.environ[key] = value

# Retrieve client ID and secret from environment variables
client_id = os.getenv('SIGMA_CLIENT_ID')
client_secret = os.getenv('SIGMA_CLIENT_SECRET')

# Auth url
url = 'https://api.sigmacomputing.com/v2/auth/token'

# Rest of your payload details
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

# Make the request and print the response
response = requests.post(url, data=payload)

# Save the bearer token from the response for future API calls
bearer_token = response.json()['access_token']

if response.status_code == 200:
    print("successful auth")
else:
    print("failed auth")

successful auth


____________
### Storing the workbook id and inode id of the table I want to export
#### workbook id = 3Jl5zKTJjM37vUjNa6LLKa
#### inode id = Tr3ZmXoAlL
____________

## 2. Use the export API to export this table
From this endpoint, I will get a Query ID that I will use to actually download this data

I can specify how many rows I want to download in addition to the data type in the response (json here)

In [9]:
# Id of the audit log workbook
workbook_id = "3Jl5zKTJjM37vUjNa6LLKa"

url = f"https://api.sigmacomputing.com/v2/workbooks/{workbook_id}/export"
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {bearer_token}",
    "content-type": "application/json"
}
# elementId is that of the table we are looking to export
data = {
    "elementId": "Tr3ZmXoAlL",
    "format": {"type": "json"},
    "rowLimit": 2000
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    print("successfully exported table")
    query_id = response.json()['queryId']
else:
    print("error exporting table")


successfully exported table


### 3. Use the Download API to retrieve this information
Using the Query ID from above, I now call the download API to get this information

The download API can take a while and the only way to figure out if it's done is to continue calling the same endpoint with the same query id until you get a 200 status code

In [11]:
import time

response_status_code = 204
start_time = time.time()

# While loop to continue call endpoint until we get back the data. Will time out after 30 seconds
while response_status_code == 204:

    if time.time() - start_time >= 60:
        break

    url = f"https://api.sigmacomputing.com/v2/query/{query_id}/download"
    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {bearer_token}"
    }

    response = requests.get(url, headers=headers)
    response_status_code = response.status_code

    time.sleep(5)

if response_status_code == 200:
    print("successfully download")
    output = response.json()
else: 
    print("unsuccessful download")



successfully download


In [12]:
output

[{'# Opens': 612,
  'Inode Id': '0554aeec-94a7-49d5-aae2-70e7ae147ea2',
  'Object Type': 'workbook',
  'Unique Openers': 14},
 {'# Opens': 233,
  'Inode Id': '12320af2-4ba5-4afa-b2f7-59b6287145d5',
  'Object Type': 'workbook',
  'Unique Openers': 17},
 {'# Opens': 163,
  'Inode Id': 'bfb329e3-091f-4c6f-b74f-855235f4b194',
  'Object Type': 'workbook',
  'Unique Openers': 19},
 {'# Opens': 133,
  'Inode Id': '806099eb-a550-4c87-891c-0fa96142d77f',
  'Object Type': 'workbook',
  'Unique Openers': 12},
 {'# Opens': 114,
  'Inode Id': '084552e6-dc0e-4be8-b28a-68edf1ff1915',
  'Object Type': 'workbook',
  'Unique Openers': 3},
 {'# Opens': 69,
  'Inode Id': '6d06ed8d-0abd-444d-b343-f7fa172049c2',
  'Object Type': 'workbook',
  'Unique Openers': 4},
 {'# Opens': 59,
  'Inode Id': '8e5f6a11-5c61-4bb7-b303-7539b0cdc619',
  'Object Type': 'workbook',
  'Unique Openers': 5},
 {'# Opens': 55,
  'Inode Id': 'fdd2f269-df67-437f-bea1-b5a17c403b25',
  'Object Type': 'workbook',
  'Unique Openers': 6},